# Proof-of-Conecpt notepad

## fetching RSS feed by feedparser

In [ ]:
import feedparser

# RSS feed URL
feed_url = "https://news.google.com/rss?hl=en-US&gl=US&ceid=US:en"

# Parse the RSS feed
feed = feedparser.parse(feed_url)

# Loop through each post
for post in feed.entries:
    print(f"{post.title}")


In [1]:
from app.services.rss_crawler import RssCrawler

crawler = RssCrawler(base_url="https://feeds.feedburner.com/geeknews-feed")
result = crawler.fetch_documents()

for element in result:
    print(f'[{element.title}]')
    print(element.content)
    print(element.href)

[구글 스칼라 PDF 리더]
구글 스칼라 PDF 리더
https://news.hada.io/user?id=neo
[올드보이 개발자가 쓴 비전 프로 후기 시리즈]
 공간 컴퓨팅은 모바일 컴퓨팅의 미래인가  2027년경에나 비전 프로가 보편화 될 것   아이폰 그 다음 새로운 희망  2007년 아이폰 출시 당시 앱 스토어 없었음, 웹 앱 중심의 환경 2008년 iOS SDK 출시로 모바일 컴퓨팅 분야가 확립 "이동성과 연결성"...
https://news.hada.io/user?id=xguru
[베네딕트 에반스의 애플 Vision Pro 한달 사용기]
 비전 프로는 놀라운 제품이지만, VR 및 AR과 마찬가지로 대중 시장에 도달하기까지는 아직 수년이 걸릴 것으로 보임 애플이 대중 시장에 도달한다 해도, 그 영향력은 얼마나 클지 미지수임  메타의 오큘러스 인수 10년  메타는 10년 전에 오큘러스를 20억 달러에 인수함...
https://news.hada.io/user?id=xguru
[실시간 벡터 그래픽을 위한 'Rive Renderer'가 이제 오픈 소스로 전환]
실시간 벡터 그래픽을 위한 'Rive Renderer'가 이제 오픈 소스로 전환
https://news.hada.io/user?id=neo
[Lcl.host: 로컬 개발 환경에서 빠르고 쉬운 HTTPS 구현]
 웹 개발자들이 로컬에서 빠르게 HTTPS 환경을 구축하고 사용할 수 있는 무료 개발 도구 개발 환경에서도 프로덕션 환경과 동일한 보안 브라우저 컨텍스트를 제공받을 수 있음 혼합 콘텐츠 문제, CORS 오류, 보안 쿠키 오류 등을 미리 해결  lcl.host를 구축한 이유  ...
https://news.hada.io/user?id=xguru
[20년간의 Not Even Wrong]
 이 블로그의 첫 게시물은 20년 전에 올라왔으며, 첫 실질적인 게시물은 20년 전 내일에 해당함 블로깅이 유행하던 시절에 시작되었으며, 다른 기본 물리학 블로그들도 비슷한 시기에 시작됨. 대부분의 블로그는 활동을 

/home/gmelan-pc/src/dogdogdogdogdogdogdogdogdog/app/services/rss_crawler.py:33: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 33 of the file /home/gmelan-pc/src/dogdogdogdogdogdogdogdogdog/app/services/rss_crawler.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  content=BeautifulSoup(entries[i]["content"][0]["value"], "html").text.replace('\n', " "),


## test LLM

### transformers - gemma

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

input_text = "안녕하세요!"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=2000)
print(tokenizer.decode(outputs[0]))

### transformers - llama2-ko

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/open-llama-2-ko-7b")
model = AutoModelForCausalLM.from_pretrained("beomi/open-llama-2-ko-7b")

input_text = "안녕하세요!"
input_ids = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**input_ids, max_new_tokens=2000)
print(tokenizer.decode(outputs[0]))

## test codellama

In [ ]:
import ollama
import requests

response = requests.get("https://feeds.feedburner.com/geeknews-feed")
rss_feed: str = response.content.decode()

print("RSS feed fetched successfully.")

prompt: str = f'''
[PROBLEM]
- implement WebsiteCrawler class with given RSS feed.
- only use feedparser, DO NOT USE BEAUTIFULSOUP.

[RSS feed]
```
{rss_feed}
```

[WebsiteCrawler class]
```python
import feedparser
from app.schemas.document_dto import DocumentDTO


from abc import ABC, abstractmethod
from typing import List, Dict

class WebsiteCrawler(ABC):
    def __init__(self, base_url: str):
        self.base_url = base_url

    @abstractmethod
    def fetch_documents(self) -> List[DocumentDTO]:
        """
        fetch documents from website
        
        :return: list of documents
        """
        pass


```

[DocumentDTO class]
```python
from dataclasses import dataclass

@dataclass
class DocumentDTO:
    title: str
    content: str
    type: str
    category: str
    href: str

```
'''


prompt = '''
this is an example of RssCrawler.
```python
from app.services.base_crawler import WebsiteCrawler
from app.schemas.document_dto import DocumentDTO

from typing import List, Dict

import feedparser

from bs4 import BeautifulSoup


class RssCrawler(WebsiteCrawler):
    def __init__(self, base_url: str):
        self.base_url = base_url


    def fetch_documents(self) -> List[DocumentDTO]:
        """
        fetch documents from website
        
        :return: a list of documents
        """

        feed_dict: feedparser.FeedParserDict = feedparser.parse(self.base_url)
        element_type: str = feed_dict["feed"]["title"]
        entries: feedparser.FeedParserDict = feed_dict["entries"]

        result: List[DocumentDTO] = list()

        for i in range(len(entries)):
            result.append(
                DocumentDTO(
                    title=entries[i]["title"],
                    content=BeautifulSoup(entries[i]["content"][0]["value"], "html").text.replace('\n', " "),
                    type=element_type,
                    category="news",
                    href=entries[i]["href"]
                )
            )

        return result

```

```
<?xml version='1.0' encoding='UTF-8'?>\n<feed xmlns='http://www.w3.org/2005/Atom'>\n<title>GeekNews - 개발/기술/스타트업 뉴스 서비스</title>\n<subtitle>개발 뉴스, 기술 관련 새소식, 스타트업 정보와 노하우, 세상의 재미난 것들을 좋아하는 사람들을 위한 뉴스 사이트. 이메일 뉴스레터/트위터/슬랙 봇으로 구독 가능</subtitle>\n<link ref='alternate' type='text/html' href='https://news.hada.io' />\n<link ref='self' type='application/atom+xml' href='https://news.hada.io/rss/news' />\n<id>https://news.hada.io/rss/news</id>\n<author>\n<name>GeekNews</name>\n<email>news@hada.io</email>\n<uri>https://news.hada.io</uri>\n</author>\n<updated>2024-03-21T16:32:30+09:00</updated>\n<icon>https://social.news.hada.io/favicon.png</icon>\n<logo>https://social.news.hada.io/WideLogo500.png</logo>\n<category term='technology'/>\n<entry>\n  <title>구글 스칼라 PDF 리더</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13925' />\n  <id>https://news.hada.io/topic?id=13925</id>\n  <updated>2024-03-21T16:32:30+09:00</updated>\n  <published>2024-03-21T16:32:30+09:00</published>\n  <author>\n    <name>neo</name>\n    <uri>https://news.hada.io/user?id=neo</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[구글 스칼라 PDF 리더]]></content>\n</entry><entry>\n  <title>올드보이 개발자가 쓴 비전 프로 후기 시리즈</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13924' />\n  <id>https://news.hada.io/topic?id=13924</id>\n  <updated>2024-03-21T11:05:47+09:00</updated>\n  <published>2024-03-21T11:05:47+09:00</published>\n  <author>\n    <name>xguru</name>\n    <uri>https://news.hada.io/user?id=xguru</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<ul>\n<li>공간 컴퓨팅은 모바일 컴퓨팅의 미래인가\n<ul>\n<li>2027년경에나 비전 프로가 보편화 될 것</li>\n</ul>\n</li>\n<li>아이폰 그 다음 새로운 희망\n<ul>\n<li>2007년 아이폰 출시 당시 앱 스토어 없었음, 웹 앱 중심의 환경</li>\n<li>2008년 iOS SDK 출시로 모바일 컴퓨팅 분야가 확립 &quot;이동성과 연결성&quot;...</p>]]></content>\n</entry><entry>\n  <title>베네딕트 에반스의 애플 Vision Pro 한달 사용기</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13923' />\n  <id>https://news.hada.io/topic?id=13923</id>\n  <updated>2024-03-21T10:44:24+09:00</updated>\n  <published>2024-03-21T10:44:24+09:00</published>\n  <author>\n    <name>xguru</name>\n    <uri>https://news.hada.io/user?id=xguru</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<ul>\n<li>비전 프로는 놀라운 제품이지만, VR 및 AR과 마찬가지로 대중 시장에 도달하기까지는 아직 수년이 걸릴 것으로 보임</li>\n<li>애플이 대중 시장에 도달한다 해도, 그 영향력은 얼마나 클지 미지수임</li>\n</ul>\n<h2>메타의 오큘러스 인수 10년</h2>\n<ul>\n<li>메타는 10년 전에 오큘러스를 20억 달러에 인수함...</p>]]></content>\n</entry><entry>\n  <title>실시간 벡터 그래픽을 위한 'Rive Renderer'가 이제 오픈 소스로 전환</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13922' />\n  <id>https://news.hada.io/topic?id=13922</id>\n  <updated>2024-03-21T10:33:22+09:00</updated>\n  <published>2024-03-21T10:33:22+09:00</published>\n  <author>\n    <name>neo</name>\n    <uri>https://news.hada.io/user?id=neo</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[실시간 벡터 그래픽을 위한 'Rive Renderer'가 이제 오픈 소스로 전환]]></content>\n</entry><entry>\n  <title>Lcl.host: 로컬 개발 환경에서 빠르고 쉬운 HTTPS 구현</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13921' />\n  <id>https://news.hada.io/topic?id=13921</id>\n  <updated>2024-03-21T10:17:01+09:00</updated>\n  <published>2024-03-21T10:17:01+09:00</published>\n  <author>\n    <name>xguru</name>\n    <uri>https://news.hada.io/user?id=xguru</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<ul>\n<li>웹 개발자들이 로컬에서 빠르게 HTTPS 환경을 구축하고 사용할 수 있는 무료 개발 도구</li>\n<li>개발 환경에서도 프로덕션 환경과 동일한 보안 브라우저 컨텍스트를 제공받을 수 있음</li>\n<li>혼합 콘텐츠 문제, CORS 오류, 보안 쿠키 오류 등을 미리 해결</li>\n</ul>\n<h2>lcl.host를 구축한 이유</h2>\n<ul>\n...</p>]]></content>\n</entry><entry>\n  <title>20년간의 Not Even Wrong</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13920' />\n  <id>https://news.hada.io/topic?id=13920</id>\n  <updated>2024-03-21T10:05:49+09:00</updated>\n  <published>2024-03-21T10:05:49+09:00</published>\n  <author>\n    <name>neo</name>\n    <uri>https://news.hada.io/user?id=neo</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<ul>\n<li>이 블로그의 첫 게시물은 20년 전에 올라왔으며, 첫 실질적인 게시물은 20년 전 내일에 해당함</li>\n<li>블로깅이 유행하던 시절에 시작되었으며, 다른 기본 물리학 블로그들도 비슷한 시기에 시작됨.</li>\n<li>대부분의 블로그는 활동을 멈췄지만, Sabine Hossenfelder의 Backreaction은 예외임.</li>\n<li>일부...</p>]]></content>\n</entry><entry>\n  <title>tlm - CodeLLaMa로 구동되는 로컬 CLI 코파일럿</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13919' />\n  <id>https://news.hada.io/topic?id=13919</id>\n  <updated>2024-03-21T10:03:01+09:00</updated>\n  <published>2024-03-21T10:03:01+09:00</published>\n  <author>\n    <name>xguru</name>\n    <uri>https://news.hada.io/user?id=xguru</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<ul>\n<li>CLI에 대해서 자연어로 요청하면 명령을 생성해주는 코파일럿 도구<br />\n<code>$ tlm suggest 'list all network interfaces but only their ip addresses'</code><br />\n<code>&gt; ip addr show | grep -oP 'inet \\K[\\d.]+</code>\n</li>\n<li>API Key 나 인터넷 연결 필요없음</li>\n<li>맥/리눅스/윈도우에서...</p>]]></content>\n</entry><entry>\n  <title>파리, 공공주택에 수십억 투입하여 혼합 사회 유지</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13918' />\n  <id>https://news.hada.io/topic?id=13918</id>\n  <updated>2024-03-21T10:02:49+09:00</updated>\n  <published>2024-03-21T10:02:49+09:00</published>\n  <author>\n    <name>neo</name>\n    <uri>https://news.hada.io/user?id=neo</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<h2>파리는 어떻게 파리로 남아있는가? 공공주택에 수십억 유로 투자</h2>\n<ul>\n<li>파리의 4분의 1 주민들이 정부 소유의 주택에 살고 있으며, 이는 저소득층 파리 시민들과 그들의 사업체들이 도시에 남아있도록 하는 공격적인 계획의 일부임.</li>\n<li>파리의 새로운 공공주택 개발 프로젝트인 Îlot Saint-Germain에서...</p>]]></content>\n</entry><entry>\n  <title>CRISPR-Cas 유전자 편집을 이용해 세포 배양에서 완전히 제거된 HIV [pdf]</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13917' />\n  <id>https://news.hada.io/topic?id=13917</id>\n  <updated>2024-03-21T09:59:49+09:00</updated>\n  <published>2024-03-21T09:59:49+09:00</published>\n  <author>\n    <name>neo</name>\n    <uri>https://news.hada.io/user?id=neo</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[CRISPR-Cas 유전자 편집을 이용해 세포 배양에서 완전히 제거된 HIV [pdf]]]></content>\n</entry><entry>\n  <title>의심스러운 불연속성들 (2020)</title>\n  <link rel='alternate' type='text/html' href='https://news.hada.io/topic?id=13916' />\n  <id>https://news.hada.io/topic?id=13916</id>\n  <updated>2024-03-21T09:56:49+09:00</updated>\n  <published>2024-03-21T09:56:49+09:00</published>\n  <author>\n    <name>neo</name>\n    <uri>https://news.hada.io/user?id=neo</uri>\n  </author>\n  <content type='html' xml:lang='ko'><![CDATA[<h2>의심스러운 불연속성</h2>\n<ul>\n<li>개인이 소득을 줄이려는 이유 중 하나는 미국의 건강보험 보조금이 소득에 따라 차등 지급되기 때문임.</li>\n<li>소득이 일정 금액을 초과하면 보조금이 급격히 줄어들어, 실제로는 소득을 줄이는 것이 더 유리할 수 있음.</li>\n<li>이러한 불연속성은 사람들이 소득을 줄이도록 ...</p>]]></content>\n</entry></feed>```

```python
from app.schemas.document_dto import DocumentDTO


from abc import ABC, abstractmethod
from typing import List, Dict

class WebsiteCrawler(ABC):
    def __init__(self, base_url: str):
        self.base_url = base_url

    @abstractmethod
    def fetch_documents(self) -> List[Dict]:
        """
        fetch documents from website
        
        :return: list of documents
        """
        pass


```

```python
from dataclasses import dataclass

@dataclass
class DocumentDTO:
    title: str
    content: str
    type: str
    category: str
    href: str

```

implement WebsiteCrawler for this RSS feed, use feedparser.
'''

response = ollama.generate(
    model="codellama:13b-python",
    prompt=prompt,
    stream=True
)

print("prompt transferred to model.")

for chunk in response:
    print(chunk["response"], end="", flush=True)


## sites

In [ ]:
import requests

response = requests.get("https://feeds.feedburner.com/geeknews-feed")
rss_feed: str = response.content.decode()

print("RSS feed fetched successfully.")

rss_feed

In [1]:
from app.schemas.document_dto import DocumentDTO
from app.schemas.website_dto import WebsiteDTO

from abc import ABC, abstractmethod
from typing import List, Dict

class WebsiteCrawler(ABC):
    def __init__(self, base_url: str):
        self.base_url = base_url

    @abstractmethod
    def fetch_documents(self) -> List[DocumentDTO]:
        """
        fetch documents from website
        
        :return: 문서의 리스트
        """
        pass

In [2]:
import feedparser
import requests

class NewsHadaWebsiteCrawler(WebsiteCrawler):
    def fetch_documents(self) -> List[Dict]:
        """
        fetch documents from rss
        
        :return: list of documents
        """
        response = requests.get(self.base_url).text
        feed = feedparser.parse(response)
        entries = feed['entries']
        result = []
        for entry in entries:
            title = entry.title
            content = entry.content[0].value
            category = ''  # 뉴스에는 카테고리가 존재하지 않음, 제거되어도 문제 없음
            href = entry.link
            result.append(DocumentDTO(title=title, content=content, type="", category=category, href=href))
        return result

In [7]:
class GNUWebsiteCrawler(WebsiteCrawler):
    def fetch_documents(self) -> List[Dict]:
        """
        """
        response = requests.get(self.base_url).text
        feed = feedparser.parse(response)
        entries = feed['entries']
        result = []

        category = feed["channel"]["title"]

        for entry in entries:
            title = entry.title
            content = entry.content[0].value
            category = category
            href = entry.link
            result.append(DocumentDTO(title=title, content=content, type="", category=category, href=href))
        
        return result

In [ ]:
a = GNUWebsiteCrawler("http://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1126&bbsId=1028")
w = a.fetch_documents()
w

In [17]:
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1126&bbsId=1028"
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1127&bbsId=1029" # 학사
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1376&bbsId=1075" # 장학
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1131&bbsId=1032" # 학술/행사
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1132&bbsId=1033" # 외부기관
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1126&bbsId=1028" # 공지/기관
base_url = "https://www.gnu.ac.kr/main/na/ntt/selectRssFeed.do?mi=1289&bbsId=1039" # GNU 소식

base_url = "https://www.gnu.ac.kr/cse/na/ntt/selectRssFeed.do?mi=17093&bbsId=4753" # 컴퓨터공학과 공지
base_url = "https://www.gnu.ac.kr/cse/na/ntt/selectRssFeed.do?mi=17078&bbsId=4845" # 컴퓨터공학과 오픈세미나

response = requests.get(base_url).text
feed = feedparser.parse(response)
entries = feed['entries']
result = []

category = feed["channel"]["title"]

for entry in entries:
    title = entry.title
    content = entry.title_detail["value"]
    category = category
    href = entry.link
    result.append(DocumentDTO(title=title, content=content, type="", category=category, href=href))



In [19]:
requests.get("https://everytime.kr/259024")

<Response [403]>

In [21]:
# SBS 정치 뉴스
requests.get("https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=01&plink=RSSREADER").text

# SBS 경제 뉴스
requests.get("https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=02&plink=RSSREADER").text

# SBS 사회 뉴스
requests.get("https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=03&plink=RSSREADER").text


# SBS 국제, 글로벌 뉴스
requests.get("https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=08&plink=RSSREADER").text



' \r\n<rss version="2.0" xmlns:dc="http://purl.org/dc/elements/1.1/"  xmlns:content="http://purl.org/rss/1.0/modules/content/" xmlns:media="http://search.yahoo.com/mrss/" xmlns:atom="http://www.w3.org/2005/Atom">\r\n<channel>\r\n\t\r\n\t<title>경제 - SBS 뉴스 섹션</title>\r\n\t<link>\t\r\n\t\t\r\n\t\t\t\r\n\t\t\t\r\n\t\t\t\thttps://news.sbs.co.kr\r\n\t\t\t\r\n\t\t\r\n\t</link>\r\n\t<description>대한민국 뉴스의 기준, 중심을 지키는 저널리즘 SBS뉴스 - 시청자의 눈높이에 맞는, 더욱 품격 있는 뉴스를 제공합니다.</description>\r\n\t<webMaster>newsservice@sbs.co.kr(SBSi)</webMaster>\r\n\t<language>ko-KR</language>\r\n\t<lastBuildDate>Thu, 4 Apr 2024 16:45:00 +0900</lastBuildDate>\r\n\t<atom:link href="https://news.sbs.co.kr/news/SectionRssFeed.do?sectionId=02&amp;amp;plink=RSSREADER" rel="self" type="application/rss+xml" />\r\n\t<image>\r\n\t\t<url>https://img.sbs.co.kr/news/logo/ios-1024x1024.png</url>\r\n\t\t<title>경제 - SBS 뉴스 섹션</title>\r\n\t\t<link>\r\n\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\r\n\t\t\t\t\thttps://news.sbs.co.kr\r\n\t\t\t\t\r\n\t\t\t\

In [ ]:
# 연합뉴스 최신
requests.get("https://www.yonhapnewstv.co.kr/browse/feed/").text

# 연합뉴스 헤드라인
requests.get("https://www.yonhapnewstv.co.kr/category/news/headline/feed/").text

# 연합뉴스 정치
requests.get("https://www.yonhapnewstv.co.kr/category/news/politics/feed/").text

# 연합뉴스 경제
requests.get("https://www.yonhapnewstv.co.kr/category/news/economy/feed/").text

# 연합뉴스 사회
requests.get("https://www.yonhapnewstv.co.kr/category/news/society/feed/").text

# 연합뉴스 지역
requests.get("https://www.yonhapnewstv.co.kr/category/news/local/feed/").text



In [ ]:
requests.get("https://v2.velog.io/rss/").text

In [ ]:
requests.get("https://v2.velog.io/rss/@oneoneone").text

requests.get("https://v2.velog.io/rss/@tosspayments").text

requests.get("https://v2.velog.io/rss/@broccolism").text

requests.get("https://v2.velog.io/rss/@oneoneone").text


https://taking.kr/korea-it-technology-blog-list-with-rss/

In [ ]:
# a = RssCrawler("https://feeds.feedburner.com/geeknews-feed")
a = NewsHadaWebsiteCrawler("https://feeds.feedburner.com/geeknews-feed")
w = a.fetch_documents()
w